In [ ]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import torch                    # Pytorch module
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our models
import kagglehub
import shutil
import random
from torchvision import transforms
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
import copy
from tqdm import tqdm
from typing import Dict, List, Union
from google.colab import drive
from pathlib import Path

%matplotlib inline

path = kagglehub.dataset_download("abdallahalidev/plantvillage-dataset")
print("Fichiers extraits dans :", path)
data_dir = "/kaggle/input/plantvillage-dataset/plantvillage dataset/color"
os.listdir(data_dir)

Fichiers extraits dans : /kaggle/input/plantvillage-dataset


['Tomato___Late_blight',
 'Tomato___healthy',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Potato___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Early_blight',
 'Tomato___Septoria_leaf_spot',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Strawberry___Leaf_scorch',
 'Peach___healthy',
 'Apple___Apple_scab',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Bacterial_spot',
 'Apple___Black_rot',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Peach___Bacterial_spot',
 'Apple___Cedar_apple_rust',
 'Tomato___Target_Spot',
 'Pepper,_bell___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Tomato___Tomato_mosaic_virus',
 'Strawberry___healthy',
 'Apple___healthy',
 'Grape___Black_rot',
 'Potato___Early_blight',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Common_rust_',
 'Grape___Esca_(Black_Measles)',
 'Raspberry___healthy'

In [ ]:
import shutil
import random
from torchvision import transforms

# 1. Paramètres
seed = 42
random.seed(seed)
torch.manual_seed(seed)

batch_size = 32
pin = torch.cuda.is_available()
num_workers = os.cpu_count()

# 2. Dossier d'origine (celui contenant les 38 classes)
data_dir = data_dir
output_base_dir = "/content"
splits = ['train', 'val', 'test']
split_ratio = [0.7, 0.15, 0.15]


# 3. Répartition train / val / test
for cls in os.listdir(data_dir):
    cls_dir = os.path.join(data_dir, cls)
    if not os.path.isdir(cls_dir):
        continue

    images = [img for img in os.listdir(cls_dir)
              if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    n_total = len(images)
    n_train = int(split_ratio[0] * n_total)
    n_val = int(split_ratio[1] * n_total)
    n_test = n_total - n_train - n_val

    split_counts = {'train': n_train, 'val': n_val, 'test': n_test}
    split_limits = {
        'train': range(0, n_train),
        'val': range(n_train, n_train + n_val),
        'test': range(n_train + n_val, n_total)
    }

    for split in splits:
        os.makedirs(os.path.join(output_base_dir, split, cls), exist_ok=True)

    for i, img in enumerate(images):
        for split in splits:
            if i in split_limits[split]:
                src = os.path.join(cls_dir, img)
                dst = os.path.join(output_base_dir, split, cls, img)
                shutil.copyfile(src, dst)
                break  # évite de copier plusieurs fois

# 4. Transforms
# Valeurs standard pour images RGB dans [0,1]
# Si tu préfères ImageNet : change mean/std
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# Transformations pour l'entraînement (avec data augmentation)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Transformations pour la validation/test (sans augmentation)
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


# 5. Dossiers finaux
train_dir = os.path.join(output_base_dir, "train")
val_dir = os.path.join(output_base_dir, "val")
test_dir = os.path.join(output_base_dir, "test")

# 6. Chargement des datasets
train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset = ImageFolder(val_dir, transform=test_transform)
test_dataset = ImageFolder(test_dir, transform=test_transform)


# 7. Loaders
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                      num_workers=num_workers, pin_memory=pin)
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                    num_workers=num_workers, pin_memory=pin)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                     num_workers=num_workers, pin_memory=pin)

# 9. Vérification
print(f"Train: {len(train_dataset)} images")
print(f"Validation: {len(val_dataset)} images")
print(f"Test: {len(test_dataset)} images")

num_classes = len(train_dataset.classes)

Train: 37997 images
Validation: 8129 images
Test: 8179 images


In [ ]:
class SimpleCNN_V1(nn.Module):
    def __init__(self, num_classes=num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 256, 256]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 32, 128, 128]

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [B, 64, 128, 128]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 64, 64, 64]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# [B, 128, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 128, 32, 32]

            nn.Conv2d(128, 256, kernel_size=3, padding=1),# [B, 256, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2),                              # [B, 256, 16, 16]

            nn.Conv2d(256, 512, kernel_size=3, padding=1),# [B, 512, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2)                               # [B, 512, 8, 8]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                                # [B, 512*8*8]
            nn.Linear(512*8*8, 512), # couche cachée (pour augmenter la complexité du réseau un minimum)
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x


In [ ]:
class UltraDeepCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            # Block 1 -> [B, 32, 256, 256]
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 32, 128, 128]

            # Block 2 -> [B, 64, 128, 128]
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 64, 64, 64]

            # Block 3 -> [B, 128, 64, 64]
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 128, 32, 32]

            # Block 4 -> [B, 256, 32, 32]
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 256, 16, 16]

            # Block 5 -> [B, 512, 16, 16]
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 512, 8, 8]

            # Block 6 -> [B, 512, 8, 8]
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 512, 4, 4]

            # Block 7 -> [B, 512, 4, 4]
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 512, 2, 2]

            # Block 8 -> [B, 512, 2, 2]
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> [B, 512, 1, 1]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                  # -> [B, 512]
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x


In [ ]:
class UltraDeepCNN_BN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 32, 128, 128]

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 64, 64, 64]

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 128, 32, 32]

            # Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 256, 16, 16]

            # Block 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 512, 8, 8]

            # Block 6
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 512, 4, 4]

            # Block 7
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 512, 2, 2]

            # Block 8
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2)   # [B, 512, 1, 1]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                # [B, 512]
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x


In [ ]:
import torch.nn as nn

class ResidualBlock_2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.need_skip_conv = in_channels != out_channels
        if self.need_skip_conv:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.skip = nn.Identity()

    def forward(self, x):
        identity = self.skip(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return self.relu(out + identity)

class ResidualCNN_MP(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.layer1 = nn.Sequential(
            ResidualBlock_2(32, 64),
            nn.MaxPool2d(2)  # [B, 64, H/2, W/2]
        )
        self.layer2 = nn.Sequential(
            ResidualBlock_2(64, 128),
            nn.MaxPool2d(2)  # [B, 128, H/4, W/4]
        )
        self.layer3 = nn.Sequential(
            ResidualBlock_2(128, 256),
            nn.MaxPool2d(2)  # [B, 256, H/8, W/8]
        )
        self.layer4 = nn.Sequential(
            ResidualBlock_2(256, 512),
            nn.MaxPool2d(2)  # [B, 512, H/16, W/16]
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))      # [B, 512, 1, 1]

        self.classifier = nn.Sequential(
            nn.Flatten(),                             # [B, 512]
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=False):
        super().__init__()
        stride = 2 if downsample else 1

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample or in_channels != out_channels
        if self.downsample:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.skip = nn.Identity()

    def forward(self, x):
        identity = self.skip(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return self.relu(out + identity)


class ResidualCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.layers = nn.Sequential(
            ResidualBlock(32, 64, downsample=True),   # [B, 64, 128, 128]
            ResidualBlock(64, 128, downsample=True),  # [B, 128, 64, 64]
            ResidualBlock(128, 256, downsample=True), # [B, 256, 32, 32]
            ResidualBlock(256, 512, downsample=True), # [B, 512, 16, 16]
            nn.AdaptiveAvgPool2d((1, 1))              # [B, 512, 1, 1]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                             # [B, 512]
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.layers(x)
        x = self.classifier(x)
        return x


In [ ]:
class ResidualCNN_minimal(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.layers = nn.Sequential(
            ResidualBlock(32, 64, downsample=True),   # [B, 64, 128, 128]
            ResidualBlock(64, 128, downsample=True),  # [B, 128, 64, 64]
            ResidualBlock(128, 256, downsample=True), # [B, 256, 32, 32]
            nn.AdaptiveAvgPool2d((1, 1))              # [B, 512, 1, 1]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                             # [B, 256]
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.layers(x)
        x = self.classifier(x)
        return x


In [ ]:
class ResidualBlock_3(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Si les dimensions des canaux changent, on adapte le skip connection
        self.skip = nn.Identity()
        if in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return self.relu(out)


class SmallResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv_layers = nn.Sequential(
            #un premier bloc qui sert à extraire les motifs, à réduire le coût pour les prochains residual block + avoir plus de canaux (faire le test en comparant avec que des ResidualBlock ?)
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 256, 256]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B, 32, 128, 128]

            ResidualBlock_3(32, 64),
            nn.MaxPool2d(2),                             # [B, 64, 64, 64]

            ResidualBlock_3(64, 128),
            nn.MaxPool2d(2),                             # [B, 128, 32, 32]

            ResidualBlock_3(128, 256),
            nn.MaxPool2d(2),                             #  [B, 256, 16, 16]

            ResidualBlock_3(256, 512)                      # [B, 512, 16, 16]
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),                # [B, 512, 1, 1] permet d'avoir moins de paramètres + pas besoin de connaître la taille en entrée
            nn.Flatten(),                                # [B, 512]
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x


In [ ]:
import torch
from torchsummary import summary


num_classes = num_classes
input_size = (3, 256, 256)  # entrée typique pour des images RGB 256x256

# Liste des modèles à inspecter
models = {
    "SimpleCNN_V1": lambda: SimpleCNN_V1(num_classes),
    "UltraDeepCNN": lambda: UltraDeepCNN(num_classes),
    "UltraDeepCNN_BN": lambda: UltraDeepCNN_BN(num_classes),
    "SmallResNet": lambda: SmallResNet(num_classes),
    "ResidualCNN_MP": lambda: ResidualCNN_MP(num_classes),
    "ResidualCNN": lambda: ResidualCNN(num_classes),
    "ResidualCNN_minimal": lambda: ResidualCNN_minimal(num_classes),
    # Ajoute ici d'autres modèles si tu en as
}

# Boucle pour afficher le résumé de chaque modèle
for name, model_fn in models.items():
    print(f"\n{'=' * 30}\nRésumé du modèle : {name}\n{'=' * 30}")
    model = model_fn().to("cuda" if torch.cuda.is_available() else "cpu")
    summary(model, input_size)



Résumé du modèle : SimpleCNN_V1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             896
              ReLU-2         [-1, 32, 256, 256]               0
         MaxPool2d-3         [-1, 32, 128, 128]               0
            Conv2d-4         [-1, 64, 128, 128]          18,496
              ReLU-5         [-1, 64, 128, 128]               0
         MaxPool2d-6           [-1, 64, 64, 64]               0
            Conv2d-7          [-1, 128, 64, 64]          73,856
              ReLU-8          [-1, 128, 64, 64]               0
         MaxPool2d-9          [-1, 128, 32, 32]               0
           Conv2d-10          [-1, 256, 32, 32]         295,168
             ReLU-11          [-1, 256, 32, 32]               0
        MaxPool2d-12          [-1, 256, 16, 16]               0
           Conv2d-13          [-1, 512, 16, 16]       1,180,160
      

In [ ]:
def count_params(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

for name, model_fn in models.items():
    model = model_fn().to("cuda" if torch.cuda.is_available() else "cpu")
    total, trainable = count_params(model)
    print(f"{name} → total : {total:,} | entraînables : {trainable:,}")

SimpleCNN_V1 → total : 18,365,798 | entraînables : 18,365,798
UltraDeepCNN → total : 19,940,678 | entraînables : 19,940,678
UltraDeepCNN_BN → total : 19,950,790 | entraînables : 19,950,790
ResidualCNN_MP → total : 5,165,990 | entraînables : 5,165,990
ResidualCNN → total : 5,165,990 | entraînables : 5,165,990
ResidualCNN_minimal → total : 1,218,982 | entraînables : 1,218,982
